In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException 
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.keys import Keys #need to send keystrokes
from datetime import datetime 
from datetime import timedelta
from dateutil import parser
import urllib
from urllib import parse
import csv
import time
from tqdm import tqdm

In [6]:
start_date = '2019-03-07'
start_date = parser.parse(start_date)
leave_date = '2019-03-12'
leave_date = parser.parse(leave_date)

Checkin_range = [start_date+timedelta(days=i*28) for i in range(26)]
Checkout_range = [leave_date+timedelta(days=i*28) for i in range(26)]

In [10]:
# create new instance of chrome 
browser = webdriver.Chrome(executable_path='C:/webdriver/chromedriver')
SCROLL_PAUSE_TIME = 0.5

In [11]:
with open('AirBnbData.csv','w',encoding="utf-8",newline='') as csvfile:
    fieldnames = ['CheckIn','CheckOut','Has_Weekend','Title','Price','Super_host','Latitude','Longitude','EntireHomeVsRoom','Bedrooms_no','Beds_no','Bathrooms_num','Sleeping_engagment','Amenities','Accuracy_stars','Communication_stars','Cleanliness_stars','Location_stars','CheckIn_stars','Value_stars','Reviewers_no','Review']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    # how many days to loop through
    for i, (CheckIn, CheckOut) in  tqdm(enumerate(zip(Checkin_range, Checkout_range)), total = len(Checkin_range)):
        browser.get("https://www.airbnb.ca/s/homes?refinement_paths%5B%5D=%2Fhomes&query=New%20York&allow_override%5B%5D=&checkin="+CheckIn.strftime('%Y-%m-%d')+"&checkout="+CheckOut.strftime('%Y-%m-%d')+"&map_toggle=false")
        browser.maximize_window()
        last_height = browser.execute_script("return document.body.scrollHeight")# Get scroll height
        then = time.time()
        while True:
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")# Scroll down to bottom
            time.sleep(SCROLL_PAUSE_TIME) # Wait to load page
            new_height = browser.execute_script("return document.body.scrollHeight")# Calculate new scroll height and compare with last scroll height
            if new_height == last_height or time.time()-then > 60:
                break
        last_height = new_height
        cont = browser.find_element_by_xpath("//body[@class='with-new-header has-epcot-header']")
        prods = cont.find_elements_by_xpath("//meta[@itemprop='url']")
        links = [prod.get_attribute("content") for prod in prods]  
        for l in tqdm(links, total=len(links)): # 18 or 30 based on the num of the homes in the pag
            driver = webdriver.Chrome(executable_path='C:/webdriver/chromedriver')
            driver.get("https://"+l)
            #driver.maximize_window()
            tot = driver.find_element_by_xpath("//body[@class='with-new-header']")

            try:
                title = tot.find_element_by_class_name('_18hrqvin').text
            except NoSuchElementException:
                title = None    
            #try:
            #elem = wait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[@class='_59m2yxn']/img")))
            map_url = driver.find_element_by_xpath("//*[@id='neighborhood']//div[@class='_59m2yxn']/img").get_attribute("src")
            query = parse.parse_qs(parse.urlparse(map_url).query)['center'][0]
            lat = query.split(",")[0]
            long = query.split(",")[1]
            #except NoSuchElementException:
            #lat = None
            #long = None
                
            try:
                for i in range(0,5):
                    day_no = CheckIn+timedelta(days=i)
                    if(day_no.weekday()< 5):
                        has_weekend = False
                    else:
                        has_weekend = True
                        break
                Has_weekend = has_weekend
            except NoSuchElementException:
                Has_weekend = None    

            try:
                price = tot.find_element_by_class_name('_doc79r').text
            except NoSuchElementException:
                price = None
                
            try:
                EntireHomeVsRoom = tot.find_element_by_class_name("_1xxanas2").text
            except NoSuchElementException:
                EntireHomeVsRoom = None
                
            #beds_bedrooms= tot.find_elements_by_xpath("//div[@id='room']//*[contains(text(),'bed')]")
                
            try:  
                num_bedrooms = tot.find_element_by_xpath("//div[@id='room']//*[contains(text(),'bedroom')]").text
            except NoSuchElementException:
                num_bedrooms = None
                                                    
            try:                                          
                num_beds = tot.find_element_by_xpath("//*[contains(text(),'bed')][not(contains(text(), 'bedroom'))]").text
            except NoSuchElementException:
                num_beds = None
            
            try:
                num_baths= tot.find_element_by_xpath("//*[@class='_czm8crp' and contains(text(),'bath')]").text
            except NoSuchElementException:
                num_baths = None
                                                   
            try:
                sleeping_engagments = tot.find_element_by_xpath("//div[@class='_520jgb']").text
            except NoSuchElementException:
                sleeping_engagments = None 
                        
            try:
                open_amenities_window = tot.find_element_by_xpath("//button[@class='_pgfpyhv' and contains(text(),'amenities')]").send_keys("\n")
                amenities= tot.find_elements_by_xpath("//div[@aria-labelledby='dls-modal__AmenitiesModal']//div[@class='_czm8crp']")
                amenities_total=[amenities[i].text for i in range(0,len(amenities))] 
            except NoSuchElementException:
                amenities_total= None
            #close_amenities_window = tot.find_element_by_xpath("//div[@class='_152qbzi']/button[@class='_1rp5252' and @aria-label='Close']").send_keys("\n")    
            
            time.sleep(3)   
            driver.refresh()
                                                        
            try:
                reviews_group = driver.find_elements_by_xpath("//*[@class='_1iu38l3']/span")
                Accuracy_stars = reviews_group[0].get_attribute("aria-label")
                Communication_stars = reviews_group[1].get_attribute("aria-label")
                Cleanliness_stars = reviews_group[2].get_attribute("aria-label")
                Location_stars = reviews_group[3].get_attribute("aria-label")
                CheckIn_stars = reviews_group[4].get_attribute("aria-label")
                Value_stars = reviews_group[5].get_attribute("aria-label")
            except:
                Accuracy_stars = None
                Communication_stars = None
                Cleanliness_stars = None
                Location_stars = None
                CheckIn_stars = None
                Value_stars = None
                pass 
                                                                   
            try:
                review_txt = driver.find_element_by_xpath("//div[@id='reviews']").text
            except (NoSuchElementException,StaleElementReferenceException) as e:
                review_txt = None
            
            try:
                Super_host = driver.find_element_by_xpath("//*[@class='_1p3joamp' and contains(text(),'Superhos')]").text
            except (NoSuchElementException,StaleElementReferenceException) as e:
                Super_host = None
                                                      
            try:                                         
                reviewers_num = tot.find_element_by_xpath("//button[@class='_ff6jfq']//span[@class='_so3dpm2']").text
            except (NoSuchElementException,StaleElementReferenceException) as e:
                reviewers_num = None
                                                      

            writer.writerow({'CheckIn' : CheckIn,
                             'CheckOut' : CheckOut,
                             'Title': title,
                             'Price': price,
                             'Has_Weekend' : Has_weekend,
                             'Latitude' : lat,
                             'Longitude' : long,
                             'EntireHomeVsRoom' : EntireHomeVsRoom,
                             'Bedrooms_no': num_bedrooms,
                             'Beds_no': num_beds,
                             'Bathrooms_num' : num_baths,
                             'Sleeping_engagment':sleeping_engagments,
                             'Amenities': amenities_total,
                             'Accuracy_stars': Accuracy_stars,
                             'Communication_stars': Communication_stars,
                             'Cleanliness_stars' : Cleanliness_stars,
                             'Location_stars' : Location_stars,
                             'CheckIn_stars' : CheckIn_stars,
                             'Value_stars' : Value_stars,
                             'Reviewers_no': reviewers_num,
                             'Super_host': Super_host,
                             'Review':review_txt})

            time.sleep(10)
            driver.close()
            
        #for ix, (checkin, checkout) in enumerate(zip(Checkin_range[i:], Checkout_range[i:])):
        #i + = ix
        

PermissionError: [Errno 13] Permission denied: 'AirBnbData.csv'

In [9]:
l

'www.airbnb.ca/rooms/24969653?check_in=2019-03-07&check_out=2019-03-12'

In [19]:
start_date = '2019-03-07'
start_date = parser.parse(start_date)
leave_date = '2019-03-12'
leave_date = parser.parse(leave_date)

Checkin_range = [start_date+timedelta(days=i*28) for i in range(26)]
Checkout_range = [leave_date+timedelta(days=i*28) for i in range(26)]

In [20]:
for i in range(len(Checkin_range)):
    print(Checkin_range[i],Checkout_range[i])


2019-03-07 00:00:00 2019-03-12 00:00:00
2019-04-04 00:00:00 2019-04-09 00:00:00
2019-05-02 00:00:00 2019-05-07 00:00:00
2019-05-30 00:00:00 2019-06-04 00:00:00
2019-06-27 00:00:00 2019-07-02 00:00:00
2019-07-25 00:00:00 2019-07-30 00:00:00
2019-08-22 00:00:00 2019-08-27 00:00:00
2019-09-19 00:00:00 2019-09-24 00:00:00
2019-10-17 00:00:00 2019-10-22 00:00:00
2019-11-14 00:00:00 2019-11-19 00:00:00
2019-12-12 00:00:00 2019-12-17 00:00:00
2020-01-09 00:00:00 2020-01-14 00:00:00
2020-02-06 00:00:00 2020-02-11 00:00:00
2020-03-05 00:00:00 2020-03-10 00:00:00
2020-04-02 00:00:00 2020-04-07 00:00:00
2020-04-30 00:00:00 2020-05-05 00:00:00
2020-05-28 00:00:00 2020-06-02 00:00:00
2020-06-25 00:00:00 2020-06-30 00:00:00
2020-07-23 00:00:00 2020-07-28 00:00:00
2020-08-20 00:00:00 2020-08-25 00:00:00
2020-09-17 00:00:00 2020-09-22 00:00:00
2020-10-15 00:00:00 2020-10-20 00:00:00
2020-11-12 00:00:00 2020-11-17 00:00:00
2020-12-10 00:00:00 2020-12-15 00:00:00
2021-01-07 00:00:00 2021-01-12 00:00:00


In [7]:
Checkin_range = [start_date+timedelta(days=i) for i in range(366)]

In [8]:
Checkin_range

[datetime.datetime(2019, 3, 7, 0, 0),
 datetime.datetime(2019, 3, 8, 0, 0),
 datetime.datetime(2019, 3, 9, 0, 0),
 datetime.datetime(2019, 3, 10, 0, 0),
 datetime.datetime(2019, 3, 11, 0, 0),
 datetime.datetime(2019, 3, 12, 0, 0),
 datetime.datetime(2019, 3, 13, 0, 0),
 datetime.datetime(2019, 3, 14, 0, 0),
 datetime.datetime(2019, 3, 15, 0, 0),
 datetime.datetime(2019, 3, 16, 0, 0),
 datetime.datetime(2019, 3, 17, 0, 0),
 datetime.datetime(2019, 3, 18, 0, 0),
 datetime.datetime(2019, 3, 19, 0, 0),
 datetime.datetime(2019, 3, 20, 0, 0),
 datetime.datetime(2019, 3, 21, 0, 0),
 datetime.datetime(2019, 3, 22, 0, 0),
 datetime.datetime(2019, 3, 23, 0, 0),
 datetime.datetime(2019, 3, 24, 0, 0),
 datetime.datetime(2019, 3, 25, 0, 0),
 datetime.datetime(2019, 3, 26, 0, 0),
 datetime.datetime(2019, 3, 27, 0, 0),
 datetime.datetime(2019, 3, 28, 0, 0),
 datetime.datetime(2019, 3, 29, 0, 0),
 datetime.datetime(2019, 3, 30, 0, 0),
 datetime.datetime(2019, 3, 31, 0, 0),
 datetime.datetime(2019, 4, 